# **"Detección y Cuantificación del Multipath en Frecuencias L1 y L2 del Sistema GPS utilizando Análisis de Combinaciones Lineales"**

In [ ]:
!pip install -q kaleido

# **1. OBTENCIÓNDE DATOS DE OBSEVABLES ESTACION GNSS CCDIS**

**1.1 Se importan las galerías.**

Se da permiso de ejecución a CRX2RNX y se extraen las librerías que se usarán

In [ ]:
!pip install plotly kaleido
!chmod +x "CRX2RNX"
import subprocess #ejecutar procesos
import requests #conexion para extraer datos de ccdis
from getpass import getpass #para ocultar contraseña
from pathlib import Path
import plotly.graph_objects as go
import numpy as np
import gzip
import shutil
import pandas as pd
import os

**1.2 Se pide inputs de la fecha de los datos que se quiera obtener.**

In [ ]:
print("Seleccione la fecha para obtención de sus datos de la estación ARE(AREQUIPA PERÚ): ")
anio=int(input("Ingrese el año: "))
mes=int(input("Ingrese el mes: "))
dia=int(input("Ingrese el dia: "))
estacion="AREG00PER"

Seleccione la fecha para obtención de sus datos de la estación ARE(AREQUIPA PERÚ): 
Ingrese el año: 2025
Ingrese el mes: 05
Ingrese el dia: 11


**1.3 Se calcula el día del año para buscar en carpetas CCDIS.**

In [ ]:
def es_bisiesto(año):
    return (año % 4 == 0 and año % 100 != 0) or (año % 400 == 0)
dias_por_mes = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
if es_bisiesto(anio):
    dias_por_mes[1] = 29
diadelanio = sum(dias_por_mes[:mes - 1]) + dia
dia_del_anio_formateado = str(diadelanio).zfill(3)
print(dia_del_anio_formateado)

131


In [ ]:
class SessionWithHeaderRedirection(requests.Session):
    AUTH_HOST = 'urs.earthdata.nasa.gov'

    def __init__(self, username, password):
        super().__init__()
        self.auth = (username, password)

    def rebuild_auth(self, prepared_request, response):
        headers = prepared_request.headers
        original_parsed = requests.utils.urlparse(response.request.url)
        redirect_parsed = requests.utils.urlparse(prepared_request.url)
        if (original_parsed.hostname != redirect_parsed.hostname) and \
           (redirect_parsed.hostname != self.AUTH_HOST) and \
           (original_parsed.hostname != self.AUTH_HOST):
            del headers['Authorization']

**1.5 Se busca los archivos y los descarga**

In [ ]:
fecha_texto = f"{anio}-{str(mes).zfill(2)}-{str(dia).zfill(2)}"
carpeta_salida = Path(f"{estacion}/{fecha_texto}")
carpeta_salida.mkdir(parents=True, exist_ok=True)
def obtener_vinculos(anio, dia_del_anio_formateado, estacion):
    urls = []
    for hora in range(24):
        subcarpeta = f"{hora:02d}"
        for minuto in range(0, 60, 15):
            h = f"{hora:02d}"
            m = f"{minuto:02d}"
            nombre_archivo = f"{estacion}_R_{anio}{dia_del_anio_formateado}{h}{m}_15M_01S_MO.crx.gz"
            url = (f"https://cddis.nasa.gov/archive/gnss/data/highrate/"f"{anio}/{dia_del_anio_formateado}/25d/{subcarpeta}/{nombre_archivo}")
            urls.append((url, nombre_archivo))
    return urls
# ---------------------- Autenticación Earthdata ----------------------
usuario = input("Usuario Earthdata: ")
contrasena = getpass("Contraseña Earthdata: ")
session = SessionWithHeaderRedirection(usuario, contrasena)
session.headers.update({"User-Agent": "Mozilla/5.0"})
# ---------------------- Descargar archivos ----------------------
vinculos = obtener_vinculos(anio, dia_del_anio_formateado, estacion)
for url, nombre_archivo in vinculos:
    ruta_destino = carpeta_salida / nombre_archivo
    if ruta_destino.exists():
        continue
    try:
        response = session.get(url, stream=True)
        if "html" in response.headers.get("Content-Type", "") or response.status_code != 200:
            print(f"No disponible: {nombre_archivo}")
        else:
            with open(ruta_destino, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
    except Exception as e:
        print(f"Error al descargar {nombre_archivo}: {e}")

Usuario Earthdata: joseSG
Contraseña Earthdata: ··········


**1.6 Obtener RNX a partir de los comprimidos .crx.gz descargados**

Se usa el ejecutable CRX2RNX para Linux desarrollado por Hatanaka, Y.

In [ ]:
ruta_crx2rnx = "/content/CRX2RNX"
def descomprimir_crx_gz(ruta_archivo_gz):
    ruta_crx = ruta_archivo_gz.with_suffix("")
    if ruta_crx.exists():
        print(f"{ruta_crx.name} ya descomprimido.")
        return ruta_crx
    try:
        with gzip.open(ruta_archivo_gz, 'rb') as f_in:
            with open(ruta_crx, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        return ruta_crx
    except Exception as e:
        print(f"Error al descomprimir {ruta_archivo_gz.name}: {e}")
        return None
def convertir_a_rnx(ruta_crx):
    try:
        result = subprocess.run(
            [ruta_crx2rnx, "-f", ruta_crx.name],
            cwd=ruta_crx.parent,
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            for archivo in ruta_crx.parent.glob(ruta_crx.stem[:4] + "*.rnx"):
                return archivo
        else:
            print(f"Error en conversión:\n{result.stderr}")
    except Exception as e:
        print(f"Fallo al ejecutar CRX2RNX con {ruta_crx.name}: {e}")
    return None
archivos_rnx = []
for archivo_gz in sorted(carpeta_salida.glob("*.crx.gz")):
    ruta_crx = descomprimir_crx_gz(archivo_gz)
    if ruta_crx:
        ruta_rnx = convertir_a_rnx(ruta_crx)
        if ruta_rnx:
            archivos_rnx.append(ruta_rnx)